## Import libraries

In [1]:
import sys
import os

# Adiciona a pasta raiz do projeto ao sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import src.config as config


2025-07-09 13:55:59.990 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Repositories\time-series-prediction-rnn


## ETL — Extract, Transform, Load of Raw Dataset

This section is responsible for the Extraction phase of the ETL process, pulling historical financial data from multiple sources as defined in the configuration file [dataset.yaml](../configs/dataset.yaml).

- Data Storage: Each dataset is stored locally in the data/raw/ directory, organized and saved individually as .csv files for traceability and versioning.
- Data Cleaning:
  - Missing values are treated using a standard cleaning strategy.
  - Features with low variance (threshold < 0.01) are removed to reduce noise and improve modeling efficiency. In this execution, no low-variance columns were found.

In [2]:
from src import dataset

dataset.main(
    # ---- REPLACE DEFAULT AS APPROPRIATE ----
    asset = '^BVSP',
    asset_focus = 'Close',
    years = 3
    # -----------------------------------------
    )

2025-07-09 13:56:03.522 | INFO     | src.dataset:main:33 - Starting raw data loading...
2025-07-09 13:56:03.523 | INFO     | src.dataset:main:39 - Requesting information between 2022-07-10 and 2025-07-09
2025-07-09 13:56:03.529 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BOVESPA (^BVSP) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-07-09 13:56:06.571 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading S&P500 (^GSPC) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-07-09 13:56:09.077 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BITCOIN (BTC-USD) from yfinance...


...

2025-07-09 13:56:16.438 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading ACUCAR (SB=F) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-07-09 13:56:19.031 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC (11) from the Central Bank of Brazil API...
2025-07-09 13:56:23.661 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading CDI (12) from the Central Bank of Brazil API...
2025-07-09 13:56:28.452 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Anual (1178) from the Central Bank of Brazil API...
2025-07-09 13:56:33.334 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Meta_Anual (432) from the Central Bank of Brazil API...
2025-07-09 13:56:40.356 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading IPCA_Mensal (433) from the Central Bank of Brazil API...
2025-07-09 13:56:42.801 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading IGP_M_Mensal (189) from the Central Bank of Brazil API...
2025-07-09 13:56:45.258 | INFO     | src.utils.dataset.

INFO:src.utils.dataset.clean_strategy:Executing CleanMissingValues...
INFO:src.utils.dataset.clean_strategy:Executing CleanLowVariance with threshold=0.01...
INFO:src.utils.dataset.clean_strategy:Columns removed due to low variance: Index([], dtype='object')
c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:108: RuntimeWarning: invalid value encountered in divide
  msw = sswn / float(dfwn)
INFO:src.utils.dataset.clean_strategy:Columns removed by f_classif: ['INCC_Mensal']
c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d ar

            Close_^BVSP  High_^BVSP  Low_^BVSP  Open_^BVSP  Volume_^BVSP  \
2025-07-03     140928.0    141304.0   139051.0    139051.0     6082200.0   
2025-07-04     141478.0    141537.0   140597.0    140928.0     2114300.0   
2025-07-05          NaN         NaN        NaN         NaN           NaN   
2025-07-06          NaN         NaN        NaN         NaN           NaN   
2025-07-07     139490.0    141342.0   139295.0    141265.0     6117300.0   
2025-07-08     139303.0    139591.0   138770.0    139491.0     6750100.0   
2025-07-09          NaN         NaN        NaN         NaN           NaN   

            Close_^GSPC   High_^GSPC    Low_^GSPC   Open_^GSPC  Volume_^GSPC  \
2025-07-03  6279.350098  6284.649902  6246.459961  6246.459961  3.378110e+09   
2025-07-04          NaN          NaN          NaN          NaN           NaN   
2025-07-05          NaN          NaN          NaN          NaN           NaN   
2025-07-06          NaN          NaN          NaN          NaN         

## Feature Engineering — Time Series Preparation

This part of the pipeline is responsible for transforming the cleaned dataset into structured features suitable for training time series models. It includes the following key steps:
- Feature Generation: Constructs relevant features based on historical market data.
- Dataset Splitting: The dataset is split into training and testing sets using a consistent strategy to preserve temporal structure.
- Time Series Windowing: Converts the sequential data into overlapping windows, enabling the model to learn temporal dependencies.
- Saving Artifacts: Both training and testing sets are stored for reproducibility, along with the transformation pipelines applied during preprocessing.

In [3]:
from src import config
from src import features

features.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    dataset_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    train_dir = config.PROCESSED_DATA_DIR,
    test_dir = config.PROCESSED_DATA_DIR,
    targets = ["^BVSP"],
    train_size_ratio = 1,
    batch_size = 1,
    sequence_length = 20
    # -----------------------------------------
)

2025-07-09 13:57:27.454 | INFO     | src.features:main:36 - Generating features from dataset...
2025-07-09 13:57:27.476 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-07-09 13:57:27.485 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...
2025-07-09 13:57:27.502 | SUCCESS  | src.features:main:51 - Saving train features in C:\Repositories\time-series-prediction-rnn\data\processed...
2025-07-09 13:57:27.505 | SUCCESS  | src.features:main:55 - Saving test features in C:\Repositories\time-series-prediction-rnn\data\processed...
2025-07-09 13:57:27.509 | SUCCESS  | src.features:main:59 - Features generation complete.
2025-07-09 13:57:27.509 | INFO     | src.features:main:61 - Saving transformers...
2025-07-09 13:57:27.510 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-07-09 13:57:27.514 | INFO     | sr

## Modeling

The modeling process begins with loading the training dataset. Next, the base model is constructed, and both the compilation and training strategies are defined. With the pipeline structure in place, the model is trained over 100 epochs using an iterative approach to adjust the weights.

During training, key metrics are monitored, including accuracy, loss (error), validation accuracy and loss (on unseen data), as well as the learning rate.

In [4]:
from src import config
from src.modeling import train

train.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    X_path = config.PROCESSED_DATA_DIR / "X_train.npy",
    y_path = config.PROCESSED_DATA_DIR / "y_train.npy",
    # -----------------------------------------
    optimizer =  None,  # Use default optimizer
    loss = None,  # Use default loss
    metrics = None,  # Use default metrics
    # -----------------------------------------
    epochs = 2**8,    
    validation_len = 2**7,
    batch_size = 2**6,    
    # -----------------------------------------
    experiment_name = "0.0-amp-main-notebook",
)

2025-07-09 13:57:30.539 | INFO     | src.modeling.train:main:48 - Loading training dataset...
2025-07-09 13:57:30.562 | INFO     | src.modeling.train:main:56 - Selecting builder strategy...
2025-07-09 13:57:30.562 | INFO     | src.modeling.train:main:62 - Selecting compile strategy...
2025-07-09 13:57:30.562 | INFO     | src.modeling.train:main:65 - Selecting training strategy...
2025-07-09 13:57:30.562 | INFO     | src.modeling.train:main:73 - Building model training pipeline template...
2025-07-09 13:57:30.562 | INFO     | src.modeling.train:main:80 - Training model...
Epoch 1/256
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 250ms/step - loss: 0.4218 - mae: 0.7453 - mse: 1.0439 - r2_score: -0.0176 - rmse: 1.0173 - smape: 171.1780 - val_loss: 0.4002 - val_mae: 0.7065 - val_mse: 1.0802 - val_r2_score: -0.0067 - val_rmse: 1.0370 - val_smape: 175.9849 - learning_rate: 0.0010
Epoch 2/256
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.4203 - mae: 0.7426 - mse: 1.0461 - r2_score: -0.0185 - rmse: 1.018

2025/07/09 13:59:02 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


2025-07-09 13:59:02.421 | INFO     | src.utils.log.log_strategy:log_environment:57 - Logging environment details...
2025-07-09 13:59:02.442 | INFO     | src.utils.log.log_strategy:run:103 - Logging Keras model...


2025/07/09 13:59:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\SAMSUNG\AppData\Local\Temp\tmp5__5ouzi\model, flavor: keras). Fall back to return ['keras==3.9.2']. Set logging level to DEBUG to see the full traceback. 
2025/07/09 13:59:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model '2025-amp-rnn' already exists. Creating a new version of this model...
Created version '3' of model '2025-amp-rnn'.


2025-07-09 13:59:16.915 | INFO     | src.utils.log.log_strategy:run:131 - Logging final and aggregate metrics...
2025-07-09 13:59:21.600 | INFO     | src.utils.log.log_strategy:run:141 - Logging artifacts...
2025-07-09 13:59:21.957 | SUCCESS  | src.utils.log.log_strategy:run:162 - MLflow logging complete.
2025-07-09 13:59:21.961 | SUCCESS  | src.modeling.train:main:113 - Experiment logged successfully.


## Predict

In the prediction stage, the trained model is loaded along with the input data. The model then performs inference, generating predictions based on the provided data.


In [5]:
from src import config
from src.modeling import predict

predict.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv"
    ,preprocessor_path = config.PROCESSED_DATA_DIR / "preprocessor.pkl"
    ,postprocessor_path = config.PROCESSED_DATA_DIR / "postprocessor.pkl"
    ,output_path = config.PREDICTED_DATA_DIR / "dataset_report.csv"
    ,model_path = config.MODELS_DIR / "default_model.keras"  # Select the best model  
    # -----------------------------------------
)

2025-07-09 13:59:21.978 | INFO     | src.modeling.predict:main:28 - Performing inference for model...
2025-07-09 13:59:22.274 | INFO     | src.modeling.predict:main:43 - Input data shape: (20, 13)
2025-07-09 13:59:22.280 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...


c:\Repositories\time-series-prediction-rnn\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
            Close_^BVSP  Volume_^BVSP       type
2025-07-07    -0.014052      1.893298       True
2025-07-08    -0.001341      0.103444       True
2025-07-09     0.001660     -0.210624  Predicted
2025-07-09 13:59:24.557 | SUCCESS  | src.modeling.predict:main:78 - Inference complete.


## Plot

Open the [Power BI Report]((../reports/pbi/amp-fynance.pbip)) and refresh data.